## **DEADLINE 18 JUNI**

In [14]:
import pandas as pd
from google.colab import files
 
 
uploaded = files.upload()

Saving Data_Plane.csv to Data_Plane.csv


In [15]:
import pandas as pd
import io
 
df = pd.read_csv(io.BytesIO(uploaded['Data_Plane.csv']))
df.head()

,Airline;Date_of_Journey;Source;Destination;Route;Dep_Time;Arrival_Time;Duration;Total_Stops;Additional_Info;Price
0,IndiGo;24/03/2019;Banglore;New Delhi;BLR _ DEL...
1,Air India;1/05/2019;Kolkata;Banglore;CCU _ IXR...
2,Jet Airways;9/06/2019;Delhi;Cochin;DEL _ LKO _...
3,IndiGo;12/05/2019;Kolkata;Banglore;CCU _ NAG _...
4,IndiGo;01/03/2019;Banglore;New Delhi;BLR _ NAG...


HTTPError: ignored